#**Drug_model_nn_JAX**

In [3]:
import jax
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib as mpl
import optax
import time
import pandas as pd
# import jaxopt
from scipy.integrate import odeint



30,000 itteration

Glucose erros

In [12]:
import numpy as np

# True values
true_values = np.array([
    [0.2, 6, 100, 209, 6.6],
    [0.2, 6, 100, 209, 6.6],
    [0.2, 6, 100, 209, 6.6],
    [0.2, 6, 100, 209, 6.6],
    [0.2, 6, 100, 209, 6.6]
])

# Predicted values
predicted_values = np.array([
    [0.19992458820343000, 6.014648914337160, 99.60297393798830, 208.43565368652300, 6.599368095397950],
    [0.20004013180732700, 5.99977970123291, 100.00370788574200, 209.0101318359380, 6.600180149078370],
    [0.20001007616519900, 6.000606060028080, 100.0037841796880, 208.97857666015600, 6.599857807159420],
    [0.19996944069862400, 5.999897003173830, 100.01251220703100, 209.0076446533200, 6.600142955780030],
    [0.19998203217983200, 6.000692367553710, 100.00099182128900, 208.97718811035200, 6.599888801574710]
])

# Create a function to compute absolute errors
def compute_absolute_error(true_values, predicted_values):
    # Compute the absolute error matrix
    absolute_error = np.abs(true_values - predicted_values)
    return absolute_error

# Compute absolute errors for each set of predicted values
absolute_errors = compute_absolute_error(true_values, predicted_values)

# Format the absolute errors in .2e format
formatted_errors = [" ".join([f"{value:.2e}" for value in row]) for row in absolute_errors]

# Display the formatted absolute errors
for i, error in enumerate(formatted_errors, start=1):
    print(f"{i}. {error}")


1. 7.54e-05 1.46e-02 3.97e-01 5.64e-01 6.32e-04
2. 4.01e-05 2.20e-04 3.71e-03 1.01e-02 1.80e-04
3. 1.01e-05 6.06e-04 3.78e-03 2.14e-02 1.42e-04
4. 3.06e-05 1.03e-04 1.25e-02 7.64e-03 1.43e-04
5. 1.80e-05 6.92e-04 9.92e-04 2.28e-02 1.11e-04


error ft gt

In [4]:
#Defining the model
def glucose_insulin_model(
    t,
    meal_t,
    meal_q,
    Vp=3,
    Vi=11,
    Vg=10,
    E=0.2,
    tp=6,
    ti=100,
    td=12,
    k=1 / 120,
    Rm=209,
    a1=6.6,
    C1=300,
    C2=144,
    C3=100,
    C4=80,
    C5=26,
    Ub=72,
    U0=4,
    Um=90,
    Rg=180,
    alpha=7.5,
    beta=1.772,
):
    def func(y, t):
        f1 = Rm / (1 + np.exp(-y[2] / Vg / C1 + a1))
        f2 = Ub * (1 - np.exp(-y[2] / Vg / C2))
        kappa = (1 / Vi + 1 / E / ti) / C4
        f3 = (U0 + Um / (1 + (kappa * y[1]) ** (-beta))) / Vg / C3
        f4 = Rg / (1 + np.exp(alpha * (y[5] / Vp / C5 - 1)))
        IG = np.sum(
            meal_q * k * np.exp(k * (meal_t - t)) *
            np.heaviside(t - meal_t, 0.5)
        )
        tmp = E * (y[0] / Vp - y[1] / Vi)
        return [
            f1 - tmp - y[0] / tp,
            tmp - y[1] / ti,
            f4 + IG - f2 - f3 * y[2],
            (y[0] - y[3]) / td,
            (y[3] - y[4]) / td,
            (y[4] - y[5]) / td,
        ]

    # defining initial conditions for 6 state variables
    Vp0, Vi0, Vg0 = 3, 11, 10
    y0 = [12 * Vp0, 4 * Vi0, 110 * Vg0 ** 2, 0, 0, 0]
    return odeint(func, y0, t)

meal_t = np.array([300, 650, 1100, 2000])
meal_q = np.array([60e3, 40e3, 50e3, 100e3])

# meal_t = np.array([300, 650])
# meal_q = np.array([60e3, 40e3])


# t = np.arange(0, 1200, 1)[:, None]
# y = glucose_insulin_model(np.ravel(t), meal_t, meal_q)

t_dense = jnp.linspace(0, 1800, 1801)[:, None]
y_dense = glucose_insulin_model(np.ravel(t_dense),meal_t, meal_q)


In [5]:
#**10 data points**

import pandas as pd


# -------Load ft_CSV file into a Pandas DataFrame
df_360 = pd.read_csv("ft_final_360.csv")
ft_360 = df_360['ft']

df_450 = pd.read_csv("ft_final_450.csv")
ft_450 = df_450['ft']

df_600 = pd.read_csv("ft_600.csv")
ft_600 = df_600['ft']

df_900 = pd.read_csv("ft_900.csv")
ft_900 = df_900['ft']

df_1800 = pd.read_csv("ft_1800.csv")
ft_1800 = df_1800['ft']

# -------Load ft_CSV file into a Pandas DataFrame
dg_360 = pd.read_csv("gt_final_360.csv")
gt_360 = dg_360['gt']

dg_450 = pd.read_csv("gt_final_450.csv")
gt_450 = dg_450['gt']

dg_600 = pd.read_csv("gt_600.csv")
gt_600 = dg_600['gt']

dg_900 = pd.read_csv("gt_900.csv")
gt_900 = dg_900['gt']

dg_1800 = pd.read_csv("gt_1800.csv")
gt_1800 = dg_1800['gt']

#-------------------------------------------------
Vp=3
Vi=11
Vg=10
E=0.2
tp=6
ti=100
td=12
Ip_true = y_dense[:, 0]
Ii_true = y_dense[:, 1]
f_t_true = -E * (Ip_true / Vp - Ii_true / Vi) - Ip_true/tp
g_t_true =  E * (Ip_true / Vp - Ii_true / Vi) - Ii_true/ti
#-------------------------------------------------

def compute_errors(true_values, predicted_values):
    # Compute Mean Absolute Error (MAE)
    mae = np.abs(true_values - predicted_values).mean()

    # Compute Root Mean Squared Error (RMSE)
    rmse = np.sqrt(((true_values - predicted_values) ** 2).mean())

    # Compute Relative Error (RE)
    numerator = np.sqrt(np.sum((true_values - predicted_values) ** 2))
    denominator = np.sqrt(np.sum(true_values ** 2))
    re = numerator / denominator

    return mae, rmse, re

print("____________________f(t)__________________")
print("_____________360 data points______________")

mae, rmse, re  = compute_errors(f_t_true, ft_360)
print(f"(MAE): {mae:.2e}")
print(f"(RMSE): {rmse:.2e}")
print(f"(RE): {re:.2e}")

print("_____________450 data points______________")

mae, rmse, re  = compute_errors(f_t_true, ft_450)
print(f"(MAE): {mae:.2e}")
print(f"(RMSE): {rmse:.2e}")
print(f"(RE): {re:.2e}")

print("_____________600 data points______________")

mae, rmse, re  = compute_errors(f_t_true, ft_600)
print(f"(MAE): {mae:.2e}")
print(f"(RMSE): {rmse:.2e}")
print(f"(RE): {re:.2e}")

print("_____________900 data points_____________")

mae, rmse, re  = compute_errors(f_t_true, ft_900)
print(f"(MAE): {mae:.2e}")
print(f"(RMSE): {rmse:.2e}")
print(f"(RE): {re:.2e}")

print("_____________1800 data points_____________")

mae, rmse, re  = compute_errors(f_t_true, ft_1800)
print(f"(MAE): {mae:.2e}")
print(f"(RMSE): {rmse:.2e}")
print(f"(RE): {re:.2e}")




____________________f(t)__________________
_____________360 data points______________
(MAE): 1.99e-02
(RMSE): 8.42e-02
(RE): 5.08e-03
_____________450 data points______________
(MAE): 1.57e-02
(RMSE): 7.58e-02
(RE): 4.57e-03
_____________600 data points______________
(MAE): 8.57e-03
(RMSE): 6.99e-02
(RE): 4.21e-03
_____________900 data points_____________
(MAE): 8.27e-03
(RMSE): 3.80e-02
(RE): 2.29e-03
_____________1800 data points_____________
(MAE): 7.89e-03
(RMSE): 5.99e-02
(RE): 3.61e-03


In [6]:

print("____________________g(t)__________________")


print("_____________360 data points______________")

mae, rmse, re = compute_errors(g_t_true, gt_360)
print(f"(MAE): {mae:.2e}")
print(f"(RMSE): {rmse:.2e}")
print(f"(RE): {re:.2e}")

print("_____________450 data points______________")

mae, rmse, re = compute_errors(g_t_true, gt_450)
print(f"(MAE): {mae:.2e}")
print(f"(RMSE): {rmse:.2e}")
print(f"(RE): {re:.2e}")

print("_____________600 data points______________")

mae, rmse, re = compute_errors(g_t_true, gt_600)
print(f"(MAE): {mae:.2e}")
print(f"(RMSE): {rmse:.2e}")
print(f"(RE): {re:.2e}")

print("_____________900 data points_____________")

mae, rmse, re = compute_errors(g_t_true, gt_900)
print(f"(MAE): {mae:.2e}")
print(f"(RMSE): {rmse:.2e}")
print(f"(RE): {re:.2e}")

print("_____________1800 data points_____________")

mae, rmse, re = compute_errors(g_t_true, gt_1800)
print(f"(MAE): {mae:.2e}")
print(f"(RMSE): {rmse:.2e}")
print(f"(RE): {re:.2e}")


____________________g(t)__________________
_____________360 data points______________
(MAE): 5.49e-02
(RMSE): 1.32e-01
(RE): 5.14e-02
_____________450 data points______________
(MAE): 4.04e-02
(RMSE): 8.61e-02
(RE): 3.36e-02
_____________600 data points______________
(MAE): 3.61e-02
(RMSE): 9.18e-02
(RE): 3.58e-02
_____________900 data points_____________
(MAE): 3.77e-02
(RMSE): 7.77e-02
(RE): 3.03e-02
_____________1800 data points_____________
(MAE): 3.14e-02
(RMSE): 6.02e-02
(RE): 2.35e-02


In [7]:
df_t = pd.read_csv("ft_t.csv")
ft_t = df_t['ft']

dg_t = pd.read_csv("gt_t.csv")
gt_t = dg_t['gt']

#-------------------------------------------------
Vp=3
Vi=11
Vg=10
E=0.2
tp=6
ti=100
td=12
Ip_true = y_dense[:, 0]
Ii_true = y_dense[:, 1]
f_t_true = -E * (Ip_true / Vp - Ii_true / Vi) - Ip_true/tp
g_t_true =  E * (Ip_true / Vp - Ii_true / Vi) - Ii_true/ti
#-------------------------------------------------

def compute_errors(true_values, predicted_values):
    # Compute Mean Absolute Error (MAE)
    mae = np.abs(true_values - predicted_values).mean()

    # Compute Root Mean Squared Error (RMSE)
    rmse = np.sqrt(((true_values - predicted_values) ** 2).mean())

    # Compute Relative Error (RE)
    numerator = np.sqrt(np.sum((true_values - predicted_values) ** 2))
    denominator = np.sqrt(np.sum(true_values ** 2))
    re = numerator / denominator

    return mae, rmse, re

print("_________900_360_______________")
print("_____________f(t)______________")

mae, rmse, re  = compute_errors(f_t_true, ft_t)
print(f"(MAE): {mae:.2e}")
print(f"(RMSE): {rmse:.2e}")
print(f"(RE): {re:.2e}")

print("_____________g(t)______________")

mae, rmse, re  = compute_errors(g_t_true, gt_t)
print(f"(MAE): {mae:.2e}")
print(f"(RMSE): {rmse:.2e}")
print(f"(RE): {re:.2e}")

FileNotFoundError: [Errno 2] No such file or directory: 'ft_t.csv'